In [1]:
import pandas as pd
import warnings
import json
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
os.chdir('../')
from model import *
os.chdir('../../data/')

In [2]:
transactions = pd.read_pickle('../data/compressed_data/transactions_train.pkl')
customers = pd.read_pickle('../data/compressed_data/customers.pkl')
articles = pd.read_pickle('../data/compressed_data/articles.pkl')

transactions['week'] = 104 - (transactions.t_dat.max() - transactions.t_dat).dt.days // 7

print('First week num: ', transactions.week.min(), '\nLast week num: ', transactions.week.max(), '\n')

First week num:  0 
Last week num:  104 



In [3]:
# Test week is week after last week in train data
test_week = transactions.week.max() + 1

# Filter transactions to last 10 weeks (most recent data)
transactions = transactions[transactions.week > transactions.week.max() - 10]

In [4]:
transactions.columns

Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id',
       'week'],
      dtype='object')

In [5]:
# Load radek's candidates
candidates_last_purchase = pd.read_csv('candidates/radek_last_purchase.csv')
candidates_bestsellers = pd.read_csv('candidates/radek_bestsellers.csv')
bestsellers_previous_week = pd.read_csv('candidates/radek_bestsellers_previous_week.csv')

# Load my candidates

In [6]:
# Load my candidates
candidates_niw_loaded = pd.read_csv('candidates_200_ranks/niw_candidates_section_name.csv')

## Submission Loop -- check best k for kaggle

In [7]:
columns_to_use = [
    'article_id', 
    'product_type_no', 
    'graphical_appearance_no', 
    'colour_group_code', 
    'perceived_colour_value_id',
    'perceived_colour_master_id', 
    'department_no', 
    'index_code',
    'index_group_no', 
    'section_no', 
    'garment_group_no', 
    'FN', 
    'Active',
    'club_member_status', 
    'fashion_news_frequency', 
    'age', 
    'postal_code', 
    'bestseller_rank',
    'not_interacted_weekly_rank'
]

model_params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'boosting_type': 'dart',
    'n_estimators': 1,
    'importance_type': 'gain'
}

In [8]:
# Get bestsellers from previous week
bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week['week'] == bestsellers_previous_week['week'].max()]['article_id'].tolist()

niw_ranks = candidates_niw_loaded[['week', 'article_id', 'not_interacted_weekly_rank']].drop_duplicates()

for k in range(20, 201, 10):
    print('k = ', k)

    # Get top k similar not bought articles for each customer
    candidates_niw = candidates_niw_loaded.groupby(['week', 'customer_id']).head(k)\
        .drop(columns=['strategy', 'not_interacted_weekly_rank'])

    candidates_niw['week'] = candidates_niw['week'] + 1

    candidates_niw['t_dat'] = '2020-07-15'
    candidates_niw['price'] = 0
    candidates_niw['sales_channel_id'] = 2

    # Prepare data for model
    train_X, train_y, test_X, test, train_baskets = prepare_data(
        transactions,
        bestsellers_previous_week,
        candidates=[candidates_last_purchase, candidates_bestsellers, candidates_niw], 
        features=[customers, articles, niw_ranks], 
        cols_to_use=columns_to_use
        )
    
    del candidates_niw
    gc.collect()
    
    # Train model
    ranker = train_model(
        train_X, 
        train_y, 
        train_baskets, 
        model_params, 
        columns_to_use, 
        show_importance=10
    )

    gc.collect()

    # Make submission
    make_submission(customers, test, test_X, ranker, bestsellers_last_week, f'submission_niw_section_name_{k}')

    del train_X, train_y, test_X, test, train_baskets, ranker
    gc.collect()



k =  20
Percentage of real transactions:  0.07133558463571564
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.551514 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1136
[LightGBM] [Info] Number of data points in the train set: 25049081, number of used features: 19
Feature importance:
bestseller_rank 0.7737461027124181
index_code 0.10011008184810608
section_no 0.0410900291955935
not_interacted_weekly_rank 0.03132290179463009
department_no 0.0236466871853341
garment_group_no 0.01416555140250633
article_id 0.0069660100041246
colour_group_code 0.004031911630184649
perceived_colour_value_id 0.002042528216514337
product_type_no 0.001973662714289394
Starting submission process...
Calculating predictions...
Done.
Creating submission...
Done.
Sav

100%|██████████| 58.1M/58.1M [00:26<00:00, 2.26MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  30
Percentage of real transactions:  0.05794373586438516
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.668405 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1133
[LightGBM] [Info] Number of data points in the train set: 31746801, number of used features: 19
Feature importance:
bestseller_rank 0.7631142918704261
index_code 0.10218001794564067
not_interacted_weekly_rank 0.0520242998617583
section_no 0.04225860935804258
garment_group_no 0.013901337951552042
department_no 0.010008765537056468
article_id 0.009598280410928991
perceived_colour_master_id 0.004503394848790901
product_type_no 0.0014927286901852339
colour_group_code 0.

100%|██████████| 58.0M/58.0M [00:27<00:00, 2.21MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  40
Percentage of real transactions:  0.04867600705824384
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.061042 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1121
[LightGBM] [Info] Number of data points in the train set: 38462608, number of used features: 19
Feature importance:
bestseller_rank 0.7441741108831169
index_code 0.1089384458629217
not_interacted_weekly_rank 0.05967289084974744
section_no 0.04747837237303363
article_id 0.01024505552961392
garment_group_no 0.010080585441331507
department_no 0.009346844030581716
product_type_no 0.003544275224060407
perceived_colour_master_id 0.002370333654259739
index_group_no 0.00169

...

Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  190
Percentage of real transactions:  0.014194768617083485
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.748705 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1131
[LightGBM] [Info] Number of data points in the train set: 140934526, number of used features: 19
Feature importance:
bestseller_rank 0.6172941368558952
index_code 0.17625329797651354
not_interacted_weekly_rank 0.07738263564327567
section_no 0.05932731469149443
garment_group_no 0.042545388143398685
department_no 0.020307688140165654
article_id 0.003913248330308298
product_type_no 0.002976290218948518
fashion_news_frequency 0.0
club_member_status 0.0
Starting submis

100%|██████████| 57.8M/57.8M [00:29<00:00, 2.08MB/s]  


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  200
Percentage of real transactions:  0.013550908453973344
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.693680 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1136
[LightGBM] [Info] Number of data points in the train set: 147799230, number of used features: 19
Feature importance:
bestseller_rank 0.6128826762680097
index_code 0.18347574470605216
not_interacted_weekly_rank 0.07498537955485089
section_no 0.05947789450547934
garment_group_no 0.04249278331822098
department_no 0.021439950378755856
product_type_no 0.0028751209111924267
article_id 0.0023704503574386616
fashion_news_frequency 0.0
club_member_status 0.0
Starting submi

100%|██████████| 57.8M/57.8M [00:27<00:00, 2.23MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.


In [10]:
!kaggle competitions submissions -c h-and-m-personalized-fashion-recommendations | head -n 21

fileName                                 date                 description                       status    publicScore  privateScore  
---------------------------------------  -------------------  --------------------------------  --------  -----------  ------------  
submission_niw_section_name_200.csv.gz   2023-12-15 00:39:35  submission_niw_section_name_200   complete  0.01349      0.01333       
submission_niw_section_name_190.csv.gz   2023-12-15 00:27:56  submission_niw_section_name_190   complete  0.01348      0.01333       
submission_niw_section_name_180.csv.gz   2023-12-15 00:16:49  submission_niw_section_name_180   complete  0.01109      0.01124       
submission_niw_section_name_170.csv.gz   2023-12-15 00:06:04  submission_niw_section_name_170   complete  0.01112      0.01127       
submission_niw_section_name_160.csv.gz   2023-12-14 23:56:06  submission_niw_section_name_160   complete  0.01351      0.01334       
submission_niw_section_name_150.csv.gz   2023-12-14 23:46:22  